<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#AB-Testing" data-toc-modified-id="AB-Testing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>AB-Testing</a></span></li></ul></div>

# AB-Testing

![cats](images/cats.jpeg)


Imaginad que somos los cientificos de datos de la empresa de videojuegos Tactile Entertainment. Los desarrolladores del juego Cookie Cats pretenden introducir un cambio en el juego para aumentar la retencion de los jugadores. En cierto nivel del juego los jugadores se encuentran una puerta que les obliga a esperar o a pagar la app. Actualmente la puerta se encuentra en nivel 30 y se pretende pasar al nivel 40, para comprobar la retencion a 1 y 7 dias. Antes de realizar el cambio definitivo en el juego se raliza un test AB.

Los datos estan alojados en `data/cookie_cats.csv`. Nuestro grupo de control sera la version actual `gate_30` y el grupo de tratamiento sera la version `gate_40`. Debemos realizar el test para 1 dia de retencion `retention_1` y para 7 dias `retention_7`.

In [2]:
# librerias

import pandas as pd
import numpy as np

from statsmodels.stats.proportion import proportions_ztest, proportion_confint
from scipy.stats import norm, sem
import scipy.stats as stats

import statsmodels.stats.api as sms
import pylab as plt
import seaborn as sns
%matplotlib inline

In [3]:
# datos
cookie=pd.read_csv('data/cookie_cats.csv')
cookie

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True
...,...,...,...,...,...
90184,9999441,gate_40,97,True,False
90185,9999479,gate_40,30,False,False
90186,9999710,gate_30,28,True,False
90187,9999768,gate_40,51,True,False


In [57]:
cookie.sum_gamerounds.mean() #la media del partidas de todos los jugadores es 51.87

51.8724567297564

In [59]:
# Seleccionamos aquellos jugadores que están por encima de la media,  jugadores habituales, y descartamos al resto.

data=cookie[(cookie['sum_gamerounds']>51.87)]
data.head(5)

,userid,version,sum_gamerounds,retention_1,retention_7
2,377,gate_40,165,True,False
4,488,gate_40,179,True,True
5,540,gate_40,187,True,True
8,1574,gate_40,108,True,True
9,1587,gate_40,153,True,False


In [60]:
data.retention_1.value_counts()

True     19255
False     3198
Name: retention_1, dtype: int64

In [61]:
data.retention_7.value_counts()

True     12455
False     9998
Name: retention_7, dtype: int64

In [62]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
userid,22453.0,5.008105e+06,2.881881e+06,377.0,2520218.0,4992735.0,7494350.0,9999441.0
sum_gamerounds,22453.0,1.659664e+02,3.673497e+02,52.0,72.0,111.0,192.0,49854.0


In [15]:
pd.crosstab(data.version, data.retention_1)

retention_1,False,True
version,,
gate_30,1507,9462
gate_40,1691,9793


In [16]:
pd.crosstab(data.version, data.retention_7)

retention_7,False,True
version,,
gate_30,4744,6225
gate_40,5254,6230


In [63]:
# muestreo (no hay usuarios repetidos en las dos versiones, con el df=data)
muestreo=data.userid.value_counts(ascending=False)
usuarios=muestreo[muestreo>1].count()
usuarios

0

In [64]:
ab_test1.version.value_counts()

gate_40    45489
gate_30    44700
Name: version, dtype: int64

In [65]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [66]:
ctrl=data[data.version=='gate_30'] #grupo de control

tto=data[data.version=='gate_40'] #grupo de tratamiento

ab_test=pd.concat([ctrl, tto], axis=0)

ab_test.reset_index(drop=True, inplace=True)

ab_test.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,2218,gate_30,305,True,True
1,2382,gate_30,73,True,False
2,2451,gate_30,204,True,True
3,2482,gate_30,108,True,True
4,2729,gate_30,105,False,True


In [24]:
ab_test.version.value_counts()

gate_40    11484
gate_30    10969
Name: version, dtype: int64

In [67]:
#visualizamos resultados para retention_1 (df=data)
tasas_conversion1=ab_test.groupby('version')['retention_1']

std=lambda x: np.std(x, ddof=0)             
error_std=lambda x: stats.sem(x, ddof=0)    

tasas_conversion1=tasas_conversion1.agg([np.mean, std, error_std])

tasas_conversion1.columns=['conv_rate', 'std', 'error_std']

tasas_conversion1.style.format('{:.3f}')

,conv_rate,std,error_std
version,,,
gate_30,0.863,0.344,0.003
gate_40,0.853,0.354,0.003


In [68]:
#visualizamos resultados para retention_7 (df=data)
tasas_conversion7=ab_test.groupby('version')['retention_7']

std=lambda x: np.std(x, ddof=0)            
error_std=lambda x: stats.sem(x, ddof=0)   

tasas_conversion7=tasas_conversion7.agg([np.mean, std, error_std])

tasas_conversion7.columns=['conv_rate', 'std', 'error_std']

tasas_conversion7.style.format('{:.3f}')

,conv_rate,std,error_std
version,,,
gate_30,0.568,0.495,0.005
gate_40,0.542,0.498,0.005


In [46]:
#testeo_retencion1 (df=data)
control_1=ab_test[ab_test.version=='gate_30']['retention_1']
tratamiento_1=ab_test[ab_test.version=='gate_40']['retention_1']

In [69]:
n_con1=control_1.count()  #contamos los Trues
n_trat1=tratamiento_1.count()

conversiones1=[control_1.sum(), tratamiento_1.sum()]  # Trues por grupo

observaciones1=[n_con1, n_trat1] # cuenta N de cada grupo

z_score, p_value = proportions_ztest(conversiones1, nobs=observaciones1)

z_score, p_value

(2.1134134411921166, 0.03456539651465957)

In [70]:
#testeo_retencion7 (df=data)
control_7=ab_test[ab_test.version=='gate_30']['retention_7']
tratamiento_7=ab_test[ab_test.version=='gate_40']['retention_7']

In [71]:
n_con7=control_7.count()  #contamos los Trues
n_trat7=tratamiento_7.count()

conversiones7=[control_7.sum(), tratamiento_7.sum()]  # Trues por grupo

observaciones7=[n_con7, n_trat7] # cuenta N de cada grupo

z_score, p_value = proportions_ztest(conversiones7, nobs=observaciones7)

z_score, p_value

(3.7699117495790033, 0.00016330528212666227)

In [72]:
#Ahora hacemos el testeo con el df original (cookie) sin eliminar aquellos jugadores con menos de 51.87 partidas

In [73]:
control_A=cookie[cookie.version=='gate_30']

tratamiento_A=cookie[cookie.version=='gate_40']

ab_test1=pd.concat([control_A, tratamiento_A], axis=0)

ab_test1.reset_index(drop=True, inplace=True)

ab_test1.head(2)

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False


In [74]:
#testeo_retencion1 (df=cookie)
control_res=ab_test1[ab_test1.version=='gate_30']['retention_1']
tratamiento_res=ab_test1[ab_test1.version=='gate_40']['retention_1']

In [75]:
n_con=control_res.count()  
n_trat=tratamiento_res.count()

conversiones=[control_res.sum(), tratamiento_res.sum()]  

observaciones=[n_con, n_trat] 
z_score, p_value = proportions_ztest(conversiones, nobs=observaciones)

z_score, p_value

(1.7840862247974725, 0.07440965529691913)

In [76]:
control_res7=ab_test1[ab_test1.version=='gate_30']['retention_7']
tratamiento_res7=ab_test1[ab_test1.version=='gate_40']['retention_7']

In [77]:
n_con=control_res7.count()   
n_trat=tratamiento_res7.count()

conversiones=[control_res7.sum(), tratamiento_res7.sum()]  

observaciones=[n_con, n_trat] 

z_score, p_value = proportions_ztest(conversiones, nobs=observaciones)

z_score, p_value

(3.164358912748191, 0.001554249975614329)

# conclusiones

1. Si tenemos en cuenta el df original (cookie) el pvalor de retention_1 es de 0.0744 y el de retention_7 es de 0.0015, por lo que sólo para retention_7 podemos rechazar la hipótesis nula H0 de que la puerta en la que empiezas a pagar esté en el nivel 30.
2. Si consideramos el df filtrado (data) con aquellos usuarios que juegan más que la media, el pvalor tanto en retention_1 como en retention_7 es< a 0.05 por lo que se puede rechazar la hipótesis H0 en los dos casos.
